In [ ]:
import pandas as pd
import pickle
import numpy as np

In [ ]:
with open('patch_responses.pickle', 'rb') as handle:
    
    data = pickle.load(handle)
    

### Create dataframe with data from PaStA 
The "pasta aggregate" script aggregates/extracts variousinformation from the mbox-result of PaStA.
The responses option extracts and dumps the data for mbox-clusters with patches and all associated
emails and commits as a pickled dictionary. These can be further
used for input to various analyses on code review.

In [ ]:
response_df = pd.DataFrame(data)

#### Some preprocessing and exploration
We explore some numbers on the input data, like patch count, commit counts, data types etc.

In [ ]:
response_df.patch_id.nunique()

In [ ]:
response_df.dtypes

In [ ]:
# Replace null/NaN patch_ids
response_df.fillna({'patch_id':'_'}, inplace=True)

In [ ]:
response_df['upstream'] = response_df['upstream'].map(list)

In [ ]:
response_df.index.name = "idx"

In [ ]:
response_df.set_index(['cluster_id', 'patch_id'], append=True, inplace=True)

### Denormalize responses
The responses column is a dict type with different attributes like mesg_id, parent (parent thread'd mesg id), and the actual message (bytestring) itself.

In [ ]:
df_melt_responses = pd.melt(response_df.responses.apply(pd.Series).reset_index(), 
            id_vars=['idx', 'cluster_id', 'patch_id'],
            value_name='responses').sort_index()

In [ ]:
df_melt_responses.drop('variable', axis=1, inplace=True)

In [ ]:
df_melt_responses.shape

In [ ]:
# This library is a wrapper around json_normalize. Due to NaNs in the columns (no responses for some patches). 
# Ideally one could also use json_normalize, but due to NaNs it would't be straightforward.
# In principle we could directly use the flat_table on the list of dicts instead of the melt step above, 
# but that somehow did not work
import flat_table

In [ ]:
df_with_responses = flat_table.normalize(df_melt_responses, expand_dicts=True, expand_lists=True)

In [ ]:
df_with_responses.drop('index', axis=1, inplace=True)

In [ ]:
df_with_responses.drop_duplicates(inplace=True)

In [ ]:
df_with_responses.to_csv("df_with_responses.csv", index=False)

In [ ]:
df_with_responses = pd.read_csv("df_with_responses.csv", index_col=['idx', 'cluster_id', 'patch_id'])

### Denormalize upstream

In [ ]:
df_melt_upstream = pd.melt(response_df.upstream.apply(pd.Series).reset_index(),
             id_vars=['idx', 'cluster_id', 'patch_id'],
             value_name='upstream').sort_index()

In [ ]:
df_melt_upstream.drop_duplicates(inplace=True)

In [ ]:
df_melt_upstream.drop('variable', axis=1, inplace=True)

In [ ]:
df_melt_upstream.head()

In [ ]:
df_melt_upstream['upstream'].nunique()

In [ ]:
df_melt_upstream.to_csv("df_with_upstream.csv", index=False)

### Merge with Dask

In [ ]:
#! python -m pip install "dask[complete]"

In [ ]:
import dask.dataframe as dd
import dask.multiprocessing
from dask.diagnostics import ProgressBar

In [ ]:
dd1 = dd.read_csv("df_with_responses.csv", blocksize=1e9, dtype={"cluster_id ": "int32", "patch_id ": "category", \
                                                                 "responses.resp_msg_id": "category", \
                                                                 "responses.parent": "category" })

In [ ]:
dd1 = dd1.set_index(['idx'])

In [ ]:
dd2 = dd.read_csv("df_with_upstream.csv", blocksize=1e9, dtype={"cluster_id ": "int32", "patch_id ": "category", \
                                                               "upstream": "category" })

In [ ]:
dd2 = dd2.set_index(['idx'])

In [ ]:
df_dask_final = dd.merge(dd1, dd2, left_index=True, right_index=True, how='left') \
.drop(['patch_id_y', 'cluster_id_y'], axis=1) \
.reset_index(drop=True) \
.rename(columns={"cluster_id_x": "cluster_id", "patch_id_x": "patch_id"})

In [ ]:
# To compute the dataframe (otherwise the computation is lazy)
df_dask_final.compute()

In [ ]:
# This can be executed directly, instead of compute above to save the frame as a single file
df_dask_final.to_csv("df_dask_final.csv", single_file = True)

In [ ]:
del df_dask_final

In [ ]:
# This is necessary if reading the final dataframe from disk. Reading with Dask gives 
# the advantage of using the resources better (blocksize parameter), dtypes are tuned to reduce memory usage.
final = dd.read_csv("df_dask_final.csv", blocksize=50e7, dtype={"cluster_id ": "int32", "patch_id ": "category", \
                                                                 "responses.resp_msg_id": "category", \
                                                                 "responses.parent": "category", \
                                                                 "upstream": "category"}).drop('Unnamed: 0', axis=1)

In [ ]:
# Convert to pandas dataframe
final = final.compute(num_workers=100) 

In [ ]:
final.shape

In [ ]:
# Apparently, duplicates can only be eliminated after converting to pandas. I suspect, while Dask is merging 
# several distributed dataframes, all duplicates cannot be detected. They are only found when the results 
# are collected as a whole
final.drop_duplicates(inplace=True)

In [ ]:
# Size considerable reduced than df_dask_final
final.shape

In [ ]:
# Save the pandas dataframe - intermediate denormalized data for the kernel patches with email response 
# and commit data
final.to_csv("df_pd_final.csv", index=False)

In [ ]:
del final

### Extract Responding author from message

In [ ]:
df_pd_final = pd.read_csv("df_pd_final.csv", index_col=False)

In [ ]:
df_pd_final.shape

In [ ]:
# Remove rows with no patch and other infos
indexNames = df_pd_final[ (df_pd_final['patch_id'] == '_') & (df_pd_final['responses.message'].isna()) & (df_pd_final['upstream'].isna())].index
df_pd_final.drop(indexNames , inplace=True)

In [ ]:
df_pd_final.shape

In [ ]:
import email
from ast import literal_eval

def try_literal_eval(s):
    try:
        return literal_eval(s)
    except ValueError:
        return s

def _get_message_field(msg, field):
    if not(np.all(pd.isnull(msg))):
        return email.message_from_bytes(msg)[field]
    else:
        return None

In [ ]:
final = dd.from_pandas(df_pd_final, npartitions=20)

In [ ]:
# This is only necessary when reading from csv, as list columns got transformed to strings
# This can probably be eliminated if we know why message is a list in the first place.
# TODO: check get_raws method from PaStA
final['responses.message'] = final['responses.message'].map(try_literal_eval)

In [ ]:
final.reset_index().compute()

In [ ]:
# TODO: check meta
final['response_author'] = final['responses.message'].map(lambda x: _get_message_field(x, 'from'), meta=pd.Series([], dtype=object, name='x'))

In [ ]:
final.compute()

#### Unique responding authors
Number of unique authors who have reviewed/commented on patches submitted for this version

In [ ]:
final['response_author'].nunique().compute(num_workers=20)

In [ ]:
final.to_csv("df_dask_final_responders.csv", single_file = True)

In [ ]:
final = dd.read_csv("df_dask_final_responders.csv", blocksize=50e7, dtype={"cluster_id ": "int32", "patch_id ": "category", \
                                                                 "responses.resp_msg_id": "category", \
                                                                 "responses.parent": "category", \
                                                                 "upstream": "category", \
                                                                 "response_author": "category"}).drop('Unnamed: 0', axis=1)

### Authors with most reviewed patch counts

In [ ]:
#! pip3 install matplotlib

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
author_patch_counts_dask = final[['patch_id', 'response_author']].groupby('response_author')['patch_id'].agg('count')

In [ ]:
author_patch_counts_dask.reset_index().compute()

In [ ]:
author_patch_counts_dask.nlargest(20).compute(). \
plot(kind='barh', stacked=False, figsize=[10,8], colormap='hsv')
plt.title('Top 20 responders by patches reviewed')
plt.ylabel('Responding authors')
plt.xlabel('Number of patch responses ')
plt.tight_layout()
plt.savefig('author_top20_patch_counts.png')
plt.savefig('author_top20_patch_counts.pdf')
plt.show()

### Upstream commits with patch counts

In [ ]:
def process_patch_ids(patch_ids):
    patch_id_list = list(patch_ids)
    try:
        patch_id_list.remove('_')
    except ValueError:
        pass
    return len(set(patch_id_list))

In [ ]:
upstream_patch_counts_dask = final[['upstream', 'patch_id']].groupby('upstream')['patch_id'].apply(process_patch_ids, meta=pd.Series([], dtype=int))


In [ ]:
final['patch_id'].nunique().compute(num_workers=20)

In [ ]:
final['upstream'].nunique().compute(num_workers=20)

In [ ]:
upstream_patch_counts_dask.nlargest(20).compute()

In [ ]:
up = upstream_patch_counts_dask.compute()

In [ ]:
up[up == 7]

In [ ]:
upstream_patch_counts_dask.nlargest(10).compute(). \
plot(kind='barh', stacked=False, figsize=[10,5], colormap='hsv')
plt.title('Top 10 upstreams by number of related patches')
plt.ylabel('Upstream commits')
plt.xlabel('Number of patches')
plt.tight_layout()
plt.savefig('upstream_top10_patch_counts.png')
plt.savefig('upstream_top10_patch_counts.pdf')
plt.show()

In [ ]:
import seaborn as sns
fig = plt.figure(figsize=(10,7))
ax = sns.distplot(upstream_patch_counts_dask.compute(), bins=40, kde=False,color='blue',\
vertical=False, kde_kws={"clip":(0,40)}, hist_kws={"range":(0, 40)})
ax.set_yscale('log')
plt.title('Upstream Distribution over patches')
plt.xlabel('patch count')
plt.ylabel('upstream')
plt.tight_layout()
plt.savefig('upstream_dist_patch_counts_40bins.png')
plt.savefig('upstream_dist_patch_counts_40bins.pdf')
plt.show()

In [ ]:
upstream_patch_counts_dask[upstream_patch_counts_dask == 6].compute()

### Upstream commits to number of responses

In [ ]:
upstream_response_counts_dask = final[['upstream', 'responses.resp_msg_id']].groupby('upstream')['responses.resp_msg_id'].agg('count')

In [ ]:
upstream_response_counts_dask.nlargest(5).compute()

In [ ]:
upstream_response_counts_dask.nlargest(15).compute(). \
plot(kind='barh', stacked=False, figsize=[10,6], colormap='Pastel1')
plt.title('Top 15 upstream commits by number of responses')
plt.ylabel('Upstream commits')
plt.xlabel('Number of patches')
plt.tight_layout()
plt.savefig('upstream_top15_response_counts.png')
plt.savefig('upstream_top15_response_counts.pdf')
plt.show()

In [ ]:
import seaborn as sns
fig = plt.figure(figsize=(10,8))
ax = sns.distplot(upstream_response_counts_dask.compute(), bins=500, kde=False,color='green',\
vertical=False, kde_kws={"clip":(0,500)}, hist_kws={"range":(0, 500)})
ax.set_yscale('log')
plt.title('Upstream Distribution over number of responses')
plt.xlabel('response count')
plt.ylabel('upstream')
plt.tight_layout()
plt.savefig('upstream_dist_response_counts_500bins.png')
plt.savefig('upstream_dist_response_counts_500bins.pdf')
plt.show()

In [ ]:
upstream_response_counts_dask[upstream_response_counts_dask == 60].count().compute()

In [ ]:
upstream_response_counts_dask[upstream_response_counts_dask == 67].compute()

In [ ]:
upstream_response_counts_dask[upstream_response_counts_dask == 52].compute()

In [ ]:
#! pip3 install seaborn

In [ ]:
fig = plt.figure(figsize=(10,8))
ax = sns.distplot(upstream_response_counts_dask.compute(), bins=200, kde=False,color='blue',\
vertical=False, kde_kws={"clip":(0,200)}, hist_kws={"range":(0,200)})
ax.set_yscale('log')
plt.title('Upstream Distribution over responses, 200 bins')
plt.xlabel('responses')
plt.ylabel('upstream count')
plt.tight_layout()
plt.savefig('upstream_dist_response_counts_200bins.png')
plt.savefig('upstream_dist_response_counts_200bins.pdf')
plt.show()

In [ ]:
fig.savefig('upstream_response_counts-0-200.png')

### Patches to responses

In [ ]:
patch_id_response_counts_dask = final[['patch_id', 'responses.resp_msg_id']].groupby('patch_id')['responses.resp_msg_id'].agg('count')

In [ ]:
fig = plt.figure(figsize=(12,12))
ax = sns.distplot(patch_id_response_counts_dask.compute(), bins=500, kde=False,color='orange',\
vertical=False, kde_kws={"clip":(0,500)}, hist_kws={"range":(0,500)})
ax.set_yscale('log')
plt.xlabel('response count')
plt.ylabel('patches')
plt.title('Patch Distribution over number of responses')
plt.tight_layout()
plt.savefig('patch_dist_response_counts_500bins.png')
plt.savefig('patch_dist_response_counts_500bins.pdf')
plt.show()

### Patch distribution by responses distinguished by with or without matching upstream

##### Patch distribution by responses without matching upstream

In [ ]:
patch_responses_upstream_df = final[['patch_id', 'upstream', 'responses.resp_msg_id']]
patch_response_without_upstream_dask = patch_responses_upstream_df[patch_responses_upstream_df['upstream'].isna() == True]



In [ ]:
patch_response_without_upstream_grouped_dask = patch_response_without_upstream_dask.groupby('patch_id')['responses.resp_msg_id'].agg('count')


In [ ]:
fig = plt.figure(figsize=(11,10))
ax = sns.distplot(patch_response_without_upstream_grouped_dask.compute(), kde=False,color='mediumvioletred',\
vertical=False)
ax.set_yscale('log')
plt.title('Patch (without matching upstream) Distribution over number of responses')
plt.xlabel('response count')
plt.ylabel('patches')
plt.tight_layout()
plt.savefig('patch_without_upstream_dist_response_counts.png')
plt.savefig('patch_without_upstream_dist_response_counts.pdf')
plt.show()

##### Patch distribution by responses with matching upstream

In [ ]:
patch_response_with_upstream_dask = patch_responses_upstream_df[patch_responses_upstream_df['upstream'].isna() == False]


In [ ]:
patch_response_with_upstream_grouped_dask = patch_response_with_upstream_dask.groupby('patch_id')['responses.resp_msg_id'].agg('count')


In [ ]:
fig = plt.figure(figsize=(11,10))
ax = sns.distplot(patch_response_with_upstream_grouped_dask.compute(), kde=False,color='olivedrab',\
vertical=False)
ax.set_yscale('log')
plt.title('Patch (with matching upstream) Distribution over number of responses')
plt.xlabel('response count')
plt.ylabel('patches')
plt.tight_layout()
plt.savefig('patch_with_upstream_dist_response_counts.png')
plt.savefig('patch_with_upstream_dist_response_counts.pdf')
plt.show()

In [ ]:
patch_response_with_upstream_grouped_dask.compute()

In [ ]:
patch_response_with_upstream_grouped_dask[patch_response_with_upstream_grouped_dask > 1000].compute()

In [ ]:
patch_response_with_upstream_grouped_dask[(patch_response_with_upstream_grouped_dask > 500) & (patch_response_with_upstream_grouped_dask < 700)].compute()